## **PySpark SQL - Getting Started Notebook**
This notebook provides an example for installing spark dependencies and a simple "getting started" syntax for PySpark SQL.

NOTE: This notebook is designed to be used with Google's Colab notebook and the Python 3 runtime.

## **Install Spark Dependencies**



In [0]:
# Install Spark dependencies
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!rm spark-2.4.5-bin-hadoop2.7.tgz
!wget --no-cookies --no-check-certificate https://downloads.apache.org/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar zxvf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install pyspark

In [0]:
!ls -al | grep spark

## **Set env variables within Pyspark**

In [0]:
# Set up required environment variables
import os
os.environ["JAVA_HOME"]  = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

## **Download Data**

In [0]:
!wget https://raw.githubusercontent.com/zaratsian/iaa_2020/master/data/bikeshare_stations.csv
!wget https://raw.githubusercontent.com/zaratsian/iaa_2020/master/data/bikeshare_trips.csv

## **Import Python and PySpark Libraries**

In [0]:
import datetime
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import monotonically_increasing_id, col, expr, when, concat, lit, udf, split
from pyspark.ml.linalg import Vectors
from pyspark.ml.regression import GBTRegressor, LinearRegression, GeneralizedLinearRegression
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorIndexer, VectorAssembler, StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

## **Initialize Spark Session**

In [0]:
spark = SparkSession.builder.appName("Bikesharing SparkSQL").master("local[*]").getOrCreate()

## **Read CSV into Spark**

In [0]:
bikeshare_trips = spark.read.load('bikeshare_trips.csv', format="csv", header=True, inferSchema=True)

## **Display first few records**

In [12]:
bikeshare_trips.show(5, truncate=False)

+----------+----------------------------------+------+-------------------+----------------+---------------------------------------+--------------+-------------------------+----------------+
|trip_id   |subscriber_type                   |bikeid|start_time         |start_station_id|start_station_name                     |end_station_id|end_station_name         |duration_minutes|
+----------+----------------------------------+------+-------------------+----------------+---------------------------------------+--------------+-------------------------+----------------+
|9900285908|Annual Membership (Austin B-cycle)|400   |10/26/2014 14:12:00|2823            |Capital Metro HQ - East 5th at Broadway|2544          |East 6th & Pedernales St.|10              |
|9900289692|Walk Up                           |248   |10/2/2015 21:12:01 |1006            |Zilker Park West                       |1008          |Nueces @ 3rd             |39              |
|9900285987|24-Hour Kiosk (Austin B-cycle)    |446

## **Execute Sample SparkSQL query**

In [14]:
bikeshare_trips.createOrReplaceTempView("bikeshare_trips")
spark.sql("SELECT subscriber_type, count(*) as count FROM bikeshare_trips group by subscriber_type order by count desc").show(truncate=False)

+----------------------------------+-----+
|subscriber_type                   |count|
+----------------------------------+-----+
|Walk Up                           |275  |
|Local365                          |115  |
|Weekender                         |34   |
|24-Hour Kiosk (Austin B-cycle)    |19   |
|Local30                           |18   |
|Annual Membership (Austin B-cycle)|14   |
|Annual                            |11   |
|Explorer                          |10   |
|Founding Member                   |2    |
|Founding Member (Austin B-cycle)  |2    |
+----------------------------------+-----+

